In [16]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [17]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm
import inspect

import model
import model_parameters as MP

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [18]:
print(f"number of campaigns {MP.C}") #70 -camp.
print(f"number of customers {MP.U}") #20.000.000 non-distinct
print(f"number of channels {MP.H}") #6-7 h. --Drop
print(f"number of planning days {MP.D}")
print(f"number of quota categories {MP.I}")
print(f"number of priority categories {MP.P}")

number of campaigns 20
number of customers 1000
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 3


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [19]:
MP.e_cu

array([[1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 1, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 0, 1, 1]])

##### - previous planning

In [20]:
MP.s_cuhd

array([[[[1, 0, 0, ..., 1, 1, 1],
         [1, 0, 1, ..., 1, 1, 0],
         [0, 0, 0, ..., 1, 0, 1]],

        [[0, 1, 0, ..., 0, 0, 1],
         [1, 1, 0, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1]],

        [[0, 0, 0, ..., 0, 0, 1],
         [0, 0, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 0, 0, 0]],

        ...,

        [[1, 0, 0, ..., 0, 0, 0],
         [0, 1, 0, ..., 0, 1, 1],
         [0, 1, 1, ..., 0, 0, 0]],

        [[1, 1, 1, ..., 1, 0, 1],
         [0, 1, 1, ..., 1, 1, 1],
         [1, 0, 0, ..., 0, 0, 1]],

        [[0, 1, 1, ..., 0, 1, 0],
         [0, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 1]]],


       [[[0, 1, 0, ..., 0, 1, 0],
         [1, 1, 0, ..., 0, 0, 0],
         [1, 1, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 1, 0, 0],
         [1, 1, 1, ..., 0, 0, 1],
         [1, 1, 0, ..., 0, 1, 0]],

        [[1, 0, 1, ..., 1, 0, 0],
         [0, 0, 1, ..., 0, 0, 0],
         [0, 1, 0, ..., 0, 0, 0]],

        ...,

        [[0, 1, 0, ..., 1, 0, 1],


##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [21]:
MP.q_ic

array([[0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [22]:
MP.rp_c

array([ 4, 98,  4,  4,  4, 22, 22,  4,  4, 98, 22,  4,  4,  4, 98, 98, 22,
       98, 22, 22])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [23]:
MP.b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [24]:
MP.k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [25]:
MP.l_c

array([4, 3, 3, 4, 2, 3, 4, 3, 3, 4, 3, 4, 4, 4, 2, 4, 4, 3, 3, 4])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [26]:
(MP.m_i, MP.n_i)

(array([4, 5, 3]), array([1, 1, 2]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [27]:
MP.t_hd

array([[700., 600., 500., 500., 700., 600., 600.],
       [500., 500., 700., 700., 600., 500., 600.],
       [500., 600., 700., 500., 600., 600., 600.]])

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [28]:
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$
$$
X_{cuhd}\le e_{cu},\quad\quad\quad\quad\forall h\in H,\forall d\in D
$$

In [29]:
print(inspect.getsource(MP.eligibility))

eligibility = lambda X, c, u, h, d: X[c,u,h,d]<=e_cu[c,u]



##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [30]:
print(inspect.getsource(MP.one_channel))

one_channel = lambda X, c, u, d: X[c,u,:,d].sum() <= 1



##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in [1]} X_{cuhd} + \sum_{h \in H}\sum_{c \in C}\sum_{d \in [2,7]}s_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in [1,2]} X_{cuhd} + \sum_{h \in H}\sum_{c \in C}\sum_{d \in [3,7]}s_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in [1,3]} X_{cuhd} + \sum_{h \in H}\sum_{c \in C}\sum_{d \in [4,7]}s_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in [1,4]} X_{cuhd} + \sum_{h \in H}\sum_{c \in C}\sum_{d \in [5,7]}s_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in [1,5]} X_{cuhd} + \sum_{h \in H}\sum_{c \in C}\sum_{d \in [6,7]}s_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in [1,6]} X_{cuhd} + \sum_{h \in H}\sum_{c \in C}\sum_{d \in [7]}s_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$


In [31]:
print(
    inspect.getsource(MP.weekly_limitation),
    inspect.getsource(MP.weekly_limitation_rh1),
    inspect.getsource(MP.weekly_limitation_rh2),
    inspect.getsource(MP.weekly_limitation_rh3),
    inspect.getsource(MP.weekly_limitation_rh4),
    inspect.getsource(MP.weekly_limitation_rh5),
    inspect.getsource(MP.weekly_limitation_rh6)
)

weekly_limitation = lambda X, u: (X[:,u,:,:]).sum() <= b
 weekly_limitation_rh = lambda f_d: lambda X, s, u : X[:,u,:,:f_d].sum() + s[:,u,:,f_d:].sum() <= b
 weekly_limitation_rh = lambda f_d: lambda X, s, u : X[:,u,:,:f_d].sum() + s[:,u,:,f_d:].sum() <= b
 weekly_limitation_rh = lambda f_d: lambda X, s, u : X[:,u,:,:f_d].sum() + s[:,u,:,f_d:].sum() <= b
 weekly_limitation_rh = lambda f_d: lambda X, s, u : X[:,u,:,:f_d].sum() + s[:,u,:,f_d:].sum() <= b
 weekly_limitation_rh = lambda f_d: lambda X, s, u : X[:,u,:,:f_d].sum() + s[:,u,:,f_d:].sum() <= b
 weekly_limitation_rh = lambda f_d: lambda X, s, u : X[:,u,:,:f_d].sum() + s[:,u,:,f_d:].sum() <= b



##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [32]:
print(inspect.getsource(MP.daily_limitation))

daily_limitation = lambda X, u, d: X[:,u,:,d].sum() <= k



##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$
$$
\sum_{d \in [1]}\sum_{h \in H}{X_{cuhd}}+ \sum_{d \in [2,7]}\sum_{h \in H}{s_{cuhd}}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$
$$
\sum_{d \in [1,2]}\sum_{h \in H}{X_{cuhd}}+ \sum_{d \in [3,7]}\sum_{h \in H}{s_{cuhd}}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$
$$
\sum_{d \in [1,3]}\sum_{h \in H}{X_{cuhd}}+ \sum_{d \in [4,7]}\sum_{h \in H}{s_{cuhd}}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$
$$
\sum_{d \in [1,4]}\sum_{h \in H}{X_{cuhd}}+ \sum_{d \in [5,7]}\sum_{h \in H}{s_{cuhd}}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$
$$
\sum_{d \in [1,5]}\sum_{h \in H}{X_{cuhd}}+ \sum_{d \in [6,7]}\sum_{h \in H}{s_{cuhd}}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$
$$
\sum_{d \in [1,6]}\sum_{h \in H}{X_{cuhd}}+ \sum_{d \in [7]}\sum_{h \in H}{s_{cuhd}}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [33]:
print(
    inspect.getsource(MP.campaign_limitation),
    inspect.getsource(MP.campaign_limitation_rh1),
    inspect.getsource(MP.campaign_limitation_rh2),
    inspect.getsource(MP.campaign_limitation_rh3),
    inspect.getsource(MP.campaign_limitation_rh4),
    inspect.getsource(MP.campaign_limitation_rh5),
    inspect.getsource(MP.campaign_limitation_rh6),
     )

campaign_limitation = lambda X, c, u: X[c,u,:,:].sum() <= l_c[c]
 campaign_limitation_rh =  lambda f_d: lambda X, s, c, u: X[c,u,:,:f_d].sum() + s[c,u,:,f_d:].sum() <=l_c[c]
 campaign_limitation_rh =  lambda f_d: lambda X, s, c, u: X[c,u,:,:f_d].sum() + s[c,u,:,f_d:].sum() <=l_c[c]
 campaign_limitation_rh =  lambda f_d: lambda X, s, c, u: X[c,u,:,:f_d].sum() + s[c,u,:,f_d:].sum() <=l_c[c]
 campaign_limitation_rh =  lambda f_d: lambda X, s, c, u: X[c,u,:,:f_d].sum() + s[c,u,:,f_d:].sum() <=l_c[c]
 campaign_limitation_rh =  lambda f_d: lambda X, s, c, u: X[c,u,:,:f_d].sum() + s[c,u,:,f_d:].sum() <=l_c[c]
 campaign_limitation_rh =  lambda f_d: lambda X, s, c, u: X[c,u,:,:f_d].sum() + s[c,u,:,f_d:].sum() <=l_c[c]



##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$
$$
\sum_{d \in [1]}\sum_{h \in H}\sum_{c \in C} {X_{cuhd} q_{ic}} + \sum_{d \in [2,7]}\sum_{h \in H}\sum_{c \in C}{s_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$
$$
\sum_{d \in [1,2]}\sum_{h \in H}\sum_{c \in C} {X_{cuhd} q_{ic}} + \sum_{d \in [3,7]}\sum_{h \in H}\sum_{c \in C}{s_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$
$$
\sum_{d \in [1,3]}\sum_{h \in H}\sum_{c \in C} {X_{cuhd} q_{ic}} + \sum_{d \in [4,7]}\sum_{h \in H}\sum_{c \in C}{s_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$
$$
\sum_{d \in [1,4]}\sum_{h \in H}\sum_{c \in C} {X_{cuhd} q_{ic}} + \sum_{d \in [5,7]}\sum_{h \in H}\sum_{c \in C}{s_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$
$$
\sum_{d \in [1,5]}\sum_{h \in H}\sum_{c \in C} {X_{cuhd} q_{ic}} + \sum_{d \in [6,7]}\sum_{h \in H}\sum_{c \in C}{s_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$
$$
\sum_{d \in [1,6]}\sum_{h \in H}\sum_{c \in C} {X_{cuhd} q_{ic}} + \sum_{d \in [7]}\sum_{h \in H}\sum_{c \in C}{s_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [34]:
print(
    inspect.getsource(MP.weekly_quota),
    inspect.getsource(MP.weekly_quota_rh1),
    inspect.getsource(MP.weekly_quota_rh2),
    inspect.getsource(MP.weekly_quota_rh3),
    inspect.getsource(MP.weekly_quota_rh4),
    inspect.getsource(MP.weekly_quota_rh5),
    inspect.getsource(MP.weekly_quota_rh6),
)

weekly_quota = lambda X, u: all((q_ic * X[:,u,:,:].sum(axis=(1,2))).sum(axis=1)<=m_i)
 weekly_quota_rh = lambda f_d :lambda X, s, u: all((q_ic * X[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s[:,u,:,f_d:].sum(axis=(1,2))).sum(axis=1) <= m_i)
 weekly_quota_rh = lambda f_d :lambda X, s, u: all((q_ic * X[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s[:,u,:,f_d:].sum(axis=(1,2))).sum(axis=1) <= m_i)
 weekly_quota_rh = lambda f_d :lambda X, s, u: all((q_ic * X[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s[:,u,:,f_d:].sum(axis=(1,2))).sum(axis=1) <= m_i)
 weekly_quota_rh = lambda f_d :lambda X, s, u: all((q_ic * X[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s[:,u,:,f_d:].sum(axis=(1,2))).sum(axis=1) <= m_i)
 weekly_quota_rh = lambda f_d :lambda X, s, u: all((q_ic * X[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s[:,u,:,f_d:].sum(axis=(1,2))).sum(axis=1) <= m_i)
 weekly_quota_rh = lambda f_d :lambda X, s, u: all((q_ic * X[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [35]:
print(inspect.getsource(MP.daily_quota))

daily_quota = lambda X, u, d: all((q_ic * X[:,u,:,d].sum(axis=(1))).sum(axis=1)<=n_i)



##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [36]:
print(inspect.getsource(MP.channel_capacity))

channel_capacity = lambda X, h, d: X[:,:,h,d].sum() <= t_hd[h,d]



In [37]:
#eligibility = lambda X, c, u, h, d: X[c,u,h,d]<=e_cu[c,u]
#channel_capacity = lambda X, h, d: X[:,:,h,d].sum() <= t_hd[h,d]
#weekly_limitation = lambda X, u: X[:,u,:,:].sum() <= b
#daily_limitation = lambda X, u, d: X[:,u,:,d].sum() <= k
#campaign_limitation = lambda X, c, u: X[c,u,:,:].sum() <= l_c[c]
#one_channel = lambda X, c, u, d: X[c,u,:,d].sum() <= 1
#weekly_quota = lambda X, u: np.all(q_ic[:,c] * X[c,u,:,:].sum()<=m_i)
#daily_quota = lambda X, u, d: np.all(q_ic[:,c] * X[c,u,:,d].sum()<=n_i)

c_i = 0
u_i = 1
h_i = 2
d_i = 3

mdl = model.Model([
    model.Constraint('eligibility',MP.eligibility, (c_i, u_i, h_i, d_i,)),
    model.Constraint('channel_capacity',MP.channel_capacity, (h_i, d_i,)),
    model.Constraint('daily_limitation',MP.daily_limitation, (u_i, d_i,)),
    model.Constraint('weekly_limitation',MP.weekly_limitation, (u_i,)),
    model.Constraint('campaign_limitation',MP.campaign_limitation, (c_i, u_i,)),
    model.Constraint('daily_quota',MP.daily_quota, (u_i, d_i,)),
    model.Constraint('weekly_quota',MP.weekly_quota, (u_i,))
], MP.objective_fn)

$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$
$$
\if{understand}
$$
$$
\begin{algorithm}[H]
 \KwData{this text}
 \KwResult{how to write algorithm with \LaTeX2e }
 initialization\;
 \While{not at end of this document}{
  read current\;
  \eIf{understand}{
   go to next section\;
   current section becomes this one\;
   }{
   go back to the beginning of current section\;
  }
 }
 \caption{How to write algorithms}
\end{algorithm}
$$

In [38]:
%%time
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')
for c in tqdm(np.argsort(-MP.rp_c), desc="Campaigns Loop"):
    for d in trange(MP.D, desc=f"Days Loop for campaign-{c}"):
        for h in range(MP.H):#trange(H, desc=f"Channels Loop at Day-{d}, Campapaign-{c}"):
            for u in range(MP.U):#trange(U, desc=f"Users Loop On Campaign-{c}"):
                X_cuhd[c,u,h,d]=1
                if not mdl.execute(X_cuhd, (c, u, h, d)):
                    X_cuhd[c,u,h,d]=0


Campaigns Loop:   0%|          | 0/20 [00:00<?, ?it/s]

Days Loop for campaign-9:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-1:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-17:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-15:   0%|          | 0/7 [00:00<?, ?it/s]

...

Days Loop for campaign-3:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-2:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-8:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-0:   0%|          | 0/7 [00:00<?, ?it/s]

CPU times: user 23.1 s, sys: 327 ms, total: 23.4 s
Wall time: 24.5 s


In [39]:
for s in mdl.stats():
    print(s)

('eligibility', 420000)
('channel_capacity', 211659)
('daily_limitation', 132095)
('weekly_limitation', 129960)
('weekly_limitation', 48472)
('campaign_limitation', 48472)
('daily_quota', 36650)
('weekly_quota', 21862)


In [40]:
display(X_cuhd.sum())
display(mdl.calc_value(X_cuhd))

6936

544910

##### - check criterion

- Sort Campaigns by rp_c
- Map every criteria to function